In [1]:
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors

In [7]:
# import os
# os.getcwd()

'/Users/useradmin/Desktop/codeology-fa19-dev'

In [9]:
path = '../movlens/'
rfilename = 'popu_ratings.csv'
mfilename = 'movies.csv'
movies = pd.read_csv(path + mfilename)
ratings = pd.read_csv(path + rfilename)

In [11]:
mov_usr_table = ratings.pivot(index='movieId', columns='userId', values='rating').fillna(0)

In [12]:
mov_usr_csr = csr_matrix(mov_usr_table)

In [13]:
title_ind_map = {movie: i
           for i, movie
           in enumerate(list(movies.set_index('movieId').loc[mov_usr_table.index].title))}
ind_title_map = {i: movie for movie, i in title_ind_map.items()}


In [40]:
def rec_movies(base_movie, k=10, metric='cosine'):
    model = NearestNeighbors(n_neighbors=k+1, algorithm='brute', metric=metric, n_jobs=-1)
    model.fit(mov_usr_csr)

    base_ind = title_ind_map[base_movie]
    recommendations = model.kneighbors(X=mov_usr_csr[base_ind], return_distance=False)[0][1:]

    rec_titles = [ind_title_map[id] for id in recommendations]
    return str(rec_titles)

In [41]:
import tabpy_client as tc
connection = tc.Client('http://localhost:9004/')
connection.deploy('recommend', rec_movies, 'Returns list of recommended movies', override=True)

In [30]:
rec_movies('Interstellar (2014)')

['The Martian (2015)',
 'Inception (2010)',
 'Django Unchained (2012)',
 'Dark Knight Rises, The (2012)',
 'Edge of Tomorrow (2014)',
 'Ex Machina (2015)',
 'Gravity (2013)',
 'The Imitation Game (2014)',
 'Mad Max: Fury Road (2015)',
 'Guardians of the Galaxy (2014)']

In [ ]:
# def rec_movies(base_movie, k=10, metric='cosine'):
#     model = NearestNeighbors(n_neighbors=k+1, algorithm='brute', metric=metric, n_jobs=-1)
#     model.fit(mov_usr_csr)

#     base_ind = title_ind_map[base_movie]
#     recommendations = model.kneighbors(X=mov_usr_csr[base_ind], return_distance=False)[0][1:]

#     rec_titles = [ind_title_map[id] for id in recommendations]
#     return rec_titles